<a href="https://colab.research.google.com/github/trungdung211/KalapaMedicalQA/blob/dataset/ViMedicalMCQA/%5Bcolab%5D-MedMCQA-Classification-Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Multiclass Text Classification**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q transformers==4.34.0 datasets==2.14.5 accelerate==0.23.0 evaluate==0.4.1 peft==0.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.1 MB/s eta 0:00:00


In [2]:
%cd /content/drive/MyDrive/MedicalMCQA-dh22

/content/drive/MyDrive/MedicalMCQA-dh22


###**1. Load Data**

In [4]:
from datasets import load_dataset
from datasets import DatasetDict

dataset = load_dataset('json', data_files='./data/qa_data_clean.jsonl')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
dataset = dataset["train"].train_test_split(test_size=0.2, shuffle=True, seed=24)

 - Single Choice: A => 0, B => 1, C => 2, D => 3, E => 4
 - Multiple Choice: A => 0, B => 1, C => 2, D => 3, E => 4, AB => 5, BC => 6,..., ABC => 10,..., ABCDE => 12.

###**2. Dataloader**

In [11]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch

letters = ["A", "B"]
# n_choices = len(letters)

# id2label = {
# }
# for i in range(1, 2**n_choices):
#     label = bin(i).replace("0b", "").rjust(5, '0')
#     id2label[i] = label

# label2id = {k: v for v, k in id2label.items()}
id2label = {
    0: "No",
    1: "Yes"
}

label2id = {
    "No": 0,
    "Yes": 1
}

num_labels = len(id2label)

def preprocess_function(examples, max_seq_length, tokenizer):
    # Tokenize the texts
    sentences = []
    labels = []
    for example in zip(examples["question"], examples["exp"],
                       examples['opa'], examples['opb'], examples['opc'], examples['opd'], examples['ope'],
                       examples['cop']):
        question = example[0]
        context = example[1]
        ops = example[2:6]
        cop = example[7]

        for i in range(len(ops)):
            answer_id = 1 if i in cop else 0

            prompt = f"<s>{context}</s>{question}</s>{ops[i]}</s>"
            sentences.append(prompt)
            labels.append(answer_id)

    model_inputs = tokenizer(
        sentences,
        padding="max_length",
        max_length=max_seq_length,
        truncation=True
    )
    model_inputs["labels"] = torch.tensor(labels)
    return model_inputs

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType

model_name = "vinai/phobert-base"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
    use_fast=True
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    # f"./model/{model_name}/checkpoint-1000",
    num_labels=num_labels
)

# peft_config = LoraConfig(
#         r=8,
#         lora_alpha=32,
#         lora_dropout=0.05,
#         target_modules=["query", "vavlue"],
#         bias="none"
#     )

# model = get_peft_model(model, peft_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from functools import partial

processed_dataset = dataset.map(
    partial(
        preprocess_function,
        max_seq_length=256,
        tokenizer=tokenizer
    ),
    batched=True,
    load_from_cache_file=False,
    remove_columns=['question', 'exp', 'cop', 'opa', 'opb', 'opc', 'opd', 'ope'],
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/10396 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/2600 [00:00<?, ? examples/s]

In [14]:
processed_dataset['train']['labels'][:10]
# processed_dataset['train']['input_ids'][0]

[0, 1, 0, 1, 1, 1, 1, 1, 0, 0]

###**3. Metric**

In [15]:
import numpy as np
import evaluate
from transformers import EvalPrediction

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred: EvalPrediction):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    result = metric.compute(predictions=predictions, references=labels)
    return result

compute_metrics = compute_metrics

###**4. Trainer**

In [23]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    f"./model/{model_name}/single",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=128,
    num_train_epochs=15,
    evaluation_strategy = "steps",
    save_strategy = "steps",
    save_steps=500,
    eval_steps=500,
    save_total_limit = 3,
    learning_rate=2e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=processed_dataset["train"],
        eval_dataset=processed_dataset["test"],
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,0.668900,0.643983,0.632404
1000,0.635300,0.613292,0.671538
1500,0.570600,0.598192,0.710481
2000,0.529800,0.533174,0.741250
2500,0.513000,0.541832,0.748558
3000,0.479800,0.514900,0.757212
3500,0.470000,0.505431,0.763750
4000,0.447200,0.514706,0.771346
4500,0.438400,0.498259,0.777981
5000,0.434800,0.507805,0.777115


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-3435b262f1ae>", line 1, in <cell line: 1>
    trainer.train()
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1591, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1984, in _inner_training_loop
    self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_keys_for_eval)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2314, in _maybe_log_save_evaluate
    self.log(logs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2692, in log
    self.control = self.callback_handler.on_log(self.args, self.state, self.control, logs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer_callback.py"

###**5. Infer**

In [3]:
%cd /content/drive/MyDrive/MedicalMCQA-dh22

/content/drive/MyDrive/MedicalMCQA-dh22


In [4]:
import pandas as pd
df = pd.read_csv("./data/public_test_with_context.csv")

In [5]:
df.head()

,id,question,option_1,option_2,option_3,option_4,option_5,option_6,context
0,level3_1,Hương đang mang thai và lo lắng mình có thể gặ...,A. Tuần 10,B.Tuần 20,C. Tuần 30,D. Tuần 40,NaN,NaN,Phương pháp chẩn đoán rau tiền đạo Thông thườn...
1,level3_2,Hương đang mang thai tuần thứ 5 và lo lắng mìn...,A. 5 tuần,B. 15 tuần,C. 25 tuần,D. 35 tuần,NaN,NaN,Phương pháp chẩn đoán rau tiền đạo Thông thườn...
2,level3_5,Có bao nhiêu loại rau tiền đạo biết rằng trong...,A. 2,B.3,C. 4,D. 5,NaN,NaN,Phương pháp chẩn đoán rau tiền đạo Thông thườn...
3,level3_13,Bệnh nhân Dũng được chuẩn đoán bị viêm gan kéo...,Có,Không,NaN,NaN,NaN,NaN,Nguyên nhân gây bệnh quai bị Quai bị có mặt tr...
4,level3_14,Một bệnh nhân bị đau tinh hoàn. Sau khi được h...,A. Nhiễm trùng đường tiết niệu,B.Ung thư tinh hoàn,C. Chấn thương,D. Giãn tĩnh mạch thừng tinh,NaN,NaN,Bị đau tinh hoàn bên phải nguy hiểm như thế nà...


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType

id2label = {
    0: "No",
    1: "Yes"
}

label2id = {
    "No": 0,
    "Yes": 1
}

num_labels = len(id2label)

# model_name = "vinai/phobert-base"
model_name = "./model/vinai/phobert-base/single/checkpoint-8000"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
    use_fast=True
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    # f"./model/{model_name}/checkpoint-1000",
    num_labels=num_labels
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
import tqdm
import torch

# convert_to_binary = {
#     0: '10000',
#     1: '01000',
#     2: '00100',
#     3: '00010',
#     4: '00001'
# }

id2Result = {
    0: [0, 0],
    1: [1, 0],
    2: [0, 1],
    3: [1, 1]
}

def remove_option_letter(choice):
    if choice[:2] in ["A.", "B.", "C.", "D.", "E.", "F."]:
        return choice[2:].strip()
    return choice

def predict_per_sample(df, model, tokenizer):
    answers = []
    for idx, row in tqdm.tqdm(df.iterrows(), total=df.shape[0]):
    # for idx, row in df.iterrows():
        context = row["context"].split("///")[0]
        question = row["question"]
        choices = [
            remove_option_letter(op) for op in row[["option_1", "option_2", "option_3", "option_4", "option_5", "option_6"]].tolist()
            if isinstance(op, str)
        ]
        # print(choices)

        answer_list = []
        for i in range(len(choices)):
            prompt = f"<s>{context}</s>{question}</s>{choices[i]}</s>"

            # model_inputs = tokenizer(prompt, return_tensors="pt")
            model_inputs = tokenizer(
                prompt,
                padding="max_length",
                max_length=256,
                truncation=True,
                return_tensors="pt"
            )
            outputs = model(**model_inputs)
            prediction = torch.argmax(outputs[0], axis=1)
            results = id2Result[prediction.item()]
            # print(results)
            answer_list.append(str(results[0]))
        answers.append("".join(answer_list))
        # print(answer_list)


        # text_choices = " ".join(choices)
        # prompt = f"Context: {context}. Question: {question}. Choice the correct answers from: {text_choices}"
        # model_inputs = tokenizer(prompt, return_tensors="pt")
        # outputs = model(**model_inputs)
        # prediction = torch.argmax(outputs[0], axis=1)
        # answer = convert_to_binary[prediction.item()]
        # answer = answer[:len(choices)]
        # answers.append(answer)
    return answers

In [12]:
model = model.to("cpu")

In [13]:
answers = predict_per_sample(df, model, tokenizer)
answers

100%|██████████| 100/100 [01:27<00:00,  1.14it/s]


['0000',
 '1000',
 '1110',
 '10',
 '1111',
 '0000',
 '0000',
 '11',
 '111',
 '11',
 '00001',
 '111',
 '111',
 '000',
 '000',
 '110',
 '1110',
 '1111',
 '111',
 '111',
 '111',
 '111',
 '111',
 '000',
 '1111',
 '10',
 '011',
 '101',
 '1111',
 '0000',
 '1111',
 '1111',
 '111',
 '111',
 '111',
 '000',
 '1111',
 '00',
 '000',
 '11',
 '111',
 '011',
 '111',
 '1111',
 '11111',
 '111',
 '111',
 '10',
 '000',
 '000',
 '11',
 '011',
 '110',
 '111',
 '111',
 '1111',
 '11',
 '111',
 '111',
 '110',
 '111',
 '1111',
 '111',
 '1111',
 '111',
 '111',
 '010',
 '101',
 '1111',
 '1111',
 '111',
 '111',
 '111',
 '1111',
 '111',
 '110',
 '1111',
 '1111',
 '1111',
 '111',
 '11111',
 '111',
 '1010',
 '1100',
 '000',
 '00000',
 '111',
 '1111',
 '111',
 '000',
 '0000',
 '1111',
 '1111',
 '1111',
 '1111',
 '1111',
 '1111',
 '0000',
 '0000',
 '000']

In [14]:
df['answer'] = answers

In [15]:
result_df = df[["id", "answer"]]

In [16]:
result_df

,id,answer
0,level3_1,0000
1,level3_2,1000
2,level3_5,1110
3,level3_13,10
4,level3_14,1111
...,...,...
95,level4_4,1111
96,level4_9,1111
97,level4_27,0000
98,level4_28,0000


In [17]:
result_df.to_csv('./data/result_phobert_classification.csv', index=False)

#Multi-label Text Classification